In [1]:
import pandas as pd
import numpy as np 

In [5]:
!pip install openpyxl

   ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/250.0 kB 660.6 kB/s eta 0:00:01
   ------------------- -------------------- 122.9/250.0 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 250.0/250.0 kB 3.1 MB/s eta 0:00:00


In [2]:
df = pd.read_excel(r'C:\Users\User\Desktop\NIR\База данных МРТ гипофиза (1).xlsx')

In [3]:
df['Файл с разметкой Ax T1+C'].isna().value_counts()

Файл с разметкой Ax T1+C
False    72
True     34
Name: count, dtype: int64

In [5]:
64+28

92

In [4]:
import os
import pandas as pd
import requests
import zipfile
import shutil
from urllib.parse import urlencode
#Assuming your dataframe is df


# Создание папки если она не существует
data_dir = 'data_29_05_24_cls108_seg72'
# os.makedirs(data_dir, exist_ok=True)

# Обход DataFrame
for index, row in df.iterrows():
    patient_id = row['ID пациента'] 
    patient_dir = f'{data_dir}/ID_{patient_id}' # путь к директории пациента

    # Создание папки для пациента, если она не существует
    os.makedirs(patient_dir, exist_ok=True)


    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    public_key = row['Местоположение файлов']  # Сюда вписываете вашу ссылку

    # Получаем загрузочную ссылку
    final_url = base_url + urlencode(dict(public_key=public_key))
    # Скачивание и разархивирование файла Если это прямые ссылки на zip файлы
    # response = requests.get(final_url, stream=True)
    response = requests.get(final_url)
    download_url = response.json()['href']
    zip_file_path = os.path.join(patient_dir, 'temp.zip') # Путь к временному zip файлу
    download_response = requests.get(download_url)

    # Сохранение zip файла
    with open(zip_file_path, 'wb') as fd:
        fd.write(download_response.content)

    # Разархивация файла
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(patient_dir)

    # Удаление временного zip файла
    os.remove(zip_file_path)
    
    # # Создание файла с метками класса
    with open(os.path.join(patient_dir, 'labels.txt'), 'w', encoding='utf-8') as f:
        f.write(f'Категория Knosp слева (S): {row["Категория Knosp слева (S)"]}\n')
        f.write(f'Категория Knosp справа (D): {row["Категория Knosp справа (D)"]}\n')

In [29]:
for index, row in df.iterrows():
    
    print(type(row["Категория Knosp слева (S)"]))


<class 'str'>
<class 'str'>
<class 'int'>
<class 'str'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'str'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>


In [18]:
import requests
from urllib.parse import urlencode

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/rfV1hVKxHwf0jg'  # Сюда вписываете вашу ссылку

# Получаем загрузочную ссылку
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']

In [19]:
download_url

'https://downloader.disk.yandex.ru/zip/24f85dccc52525bca9acc1e98b03000cc63fe5e8f72b8913379b1e324f8b930c/662e4d36/c3ZJbW5HNjFxdFFvcWREdDNxRVhtdnY0Z2RIa1JZQUQvbGtmN3hIVm00YWtyTW1RQkxBMFY1RjVKU1lLWCtQdHEvSjZicG1SeU9Kb25UM1ZvWG5EYWc9PTo=?uid=0&filename=ID89.zip&disposition=attachment&hash=svImnG61qtQoqdDt3qEXmvv4gdHkRYAD/lkf7xHVm4akrMmQBLA0V5F5JSYKX%2BPtq/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&owner_uid=433385723&tknv=v2'

In [21]:
final_url

'https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key=https%3A%2F%2Fdisk.yandex.ru%2Fd%2FzJzYIbdf_RDdUg'

In [ ]:
lables = {
    '0':0,
    '1':1,
    '2':2,
    '3a':3,
    '3b':4,
    '4':5
}

In [74]:
def get_pathes(path):
    x_pathes_all = []
    y_pathes_all = []
    for patient in os.listdir(path):
        x_pathes = []
        y_pathes = []
        
        for ID_s in os.listdir(path + '/'+ patient ):
            if 'ID' in ID_s:
                for tif_name in os.listdir(path + '/'+ patient + '/'+ID_s):
                    print(tif_name)
                    if 'Cor' in tif_name:
                        if 'T1' in tif_name:
                            if 'mask' not in tif_name.lower():
                                x_pathes.append(path + '/'+ patient + '/' + tif_name)
                                y_pathes.append(path + '/'+ patient + '/' +'labels.txt')
                            # elif 'mask' in tif_name.lower():
                            #     y_pathes.append(path + '/'+ patient + '/' + tif_name)
                        elif 'T2' in tif_name:
                            if 'mask' not in tif_name.lower():
                                x_pathes.append(path + '/'+ patient + '/' + tif_name)
                                y_pathes.append(path + '/'+ patient + '/' +'labels.txt')
                            # elif 'mask' in tif_name.lower():
                            #     y_pathes.append(path + '/'+ patient + '/' + tif_name)
                        
        x_pathes_all.append(x_pathes)
        y_pathes_all.append(y_pathes)
                
                
    return x_pathes_all, y_pathes_all
get_pathes('data_TEST')

In [75]:
get_pathes('data_TEST')

ID1_Ax T1+C_SE12.tif
ID1_Ax T1+C_SE12_MASK.tif
ID1_Cor T1+C_SE11.tif
ID1_Cor T1+C_SE11_MASK.tif
ID1_Cor T2_SE5.tif
ID1_Cor T2_SE5_MASK.tif
ID1_Sag T1+C_SE10.tif
ID1_Sag T1+C_SE10_MASK.tif
ID1_Sag T2_SE7.tif
ID1_Sag T2_SE7_MASK.tif
ID10_ Cor T1+C_SE19.tif
ID10_ Cor T1+C_SE19_MASK.tif
ID10_ Sag T1+C_SE18.tif
ID10_ Sag T1+C_SE18_MASK.tif
ID10_Ax T1+C_SE20.tif
ID10_Ax T1+C_SE20_MASK.tif
ID10_Cor T2_SE13.tif
ID10_Cor T2_SE13_MASK.tif
ID10_Sag T2_SE15.tif
ID10_Sag T2_SE15_MASK.tif
ID12_Ax T1+C_SE14.tif
ID12_Ax T1+C_SE14_MASK.tif
ID12_Cor T1+C_SE12.tif
ID12_Cor T1+C_SE12_MASK.tif
ID12_Cor T2_SE7.tif
ID12_Cor T2_SE7_MASK.tif
ID12_Sag T1+C_SE11.tif
ID12_Sag T1+C_SE11_MASK.tif
ID12_Sag T2_SE9.tif
ID12_Sag T2_SE9_MASK.tif
ID13_ Cor T1+C_SE7.tif
ID13_ Cor T1+C_SE7_MASK.tif
ID13_ Sag T1+C_SE6.tif
ID13_ Sag T1+C_SE6_MASK.tif
ID13_Ax T1+C_SE8.tif
ID13_Ax T1+C_SE8_MASK.tif
ID13_Cor T2_SE4.tif
ID13_Cor T2_SE4_MASK.tif
ID13_Sag T2_SE3.tif
ID13_Sag T2_SE3_MASK.tif
ID14_Ax T1+C_SE14.tif
ID14_Ax T1+C_SE14_

([['data_TEST/ID_1/ID1_Cor T1+C_SE11.tif',
   'data_TEST/ID_1/ID1_Cor T2_SE5.tif'],
  ['data_TEST/ID_10/ID10_ Cor T1+C_SE19.tif',
   'data_TEST/ID_10/ID10_Cor T2_SE13.tif'],
  ['data_TEST/ID_12/ID12_Cor T1+C_SE12.tif',
   'data_TEST/ID_12/ID12_Cor T2_SE7.tif'],
  ['data_TEST/ID_13/ID13_ Cor T1+C_SE7.tif',
   'data_TEST/ID_13/ID13_Cor T2_SE4.tif'],
  ['data_TEST/ID_14/ID14_Cor T1+C_SE13.tif',
   'data_TEST/ID_14/ID14_Cor T2_SE6.tif'],
  ['data_TEST/ID_15/ID15_ Cor T1+C_SE12.tif',
   'data_TEST/ID_15/ID15_Cor T2_SE8.tif'],
  ['data_TEST/ID_16/ID16_Cor T1+C_SE11.tif',
   'data_TEST/ID_16/ID16_Cor T2_SE9.tif'],
  ['data_TEST/ID_18/ID18_ Cor T1 SE+C_SE12.tif',
   'data_TEST/ID_18/ID18_Cor T2 FSE (Thin)_SE8.tif'],
  ['data_TEST/ID_2/ID2_Cor T1+C_SE8.tif', 'data_TEST/ID_2/ID2_Cor T2_SE3.tif'],
  ['data_TEST/ID_20/ID20_ Cor T1 SE+C_SE13.tif',
   'data_TEST/ID_20/ID20_Cor T2 FSE (Thin)_SE8.tif'],
  ['data_TEST/ID_21/ID21_ Cor T1 SE+C_SE14.tif',
   'data_TEST/ID_21/ID21_Cor T2 FSE (Thin)_SE5.tif